In [1]:
import numpy as np
import cvxpy as cp
import pandas as pd

load in the csvs that will be used in the optimization problem

In [2]:
df_distance_matrix = pd.read_csv("bus-to-garage.csv")

In [3]:
# remove unnessecary rows
df_distance_matrix.drop(["route_id", "route_shor", "trip_id", "trip_heads", "trip_short", "direction_", "agency_id", 
         "headway_se", "O_TERMINAL", "LEN_KM", "STD_FARE", "route_time_s", "buses", "standard_buses", "InputID"], axis=1, inplace= True)
df_distance_matrix.head()

,Al Amal,Al Amiriyyah,Al Sawah,Al-Mostaqbal,Athar an Naby,Badr,Cairo,El Mizalat,El-Mounib,El-Obour,El-Teraa El-,Fateh,Fom El-Khalig,Gesr Al Suez,Giza,Helwan,Imbaba,Maadi,Nasr,Port Said
0,18301.473900,18368.666923,18581.507345,21992.750178,2353.270354,301.835232,23032.959577,15936.860132,2660.023258,9069.280756,16572.264582,13528.851235,5649.756059,25014.708676,4709.636163,21633.618083,11516.717063,22232.238817,13351.165462,17360.939986
1,22264.117426,9331.904674,10136.511552,19878.244003,11709.129426,12022.225132,26166.785284,4638.922373,12042.965174,17404.387609,5860.814645,10757.511185,7202.795330,17255.145534,10047.655456,32522.073519,1872.545756,25098.498174,10922.431914,7431.127386
2,21041.718403,7198.050708,6578.351970,11800.828675,23003.233444,24742.272869,22156.504548,12110.315236,26080.211847,23834.582677,10744.656325,12166.493796,19540.150121,1504.971689,25068.993985,38179.638273,18363.682020,21153.031956,12466.715871,9070.618353
3,14402.720581,17883.746118,17864.535913,19103.846632,2457.931106,4216.796435,19145.063011,16591.915457,6686.557516,5059.463572,16912.100814,11702.328755,6644.090651,23612.153677,8537.047733,18801.613754,13565.257036,18387.188746,11448.449481,17310.326051
4,18863.277607,11661.020550,10730.545022,8663.670276,24952.800311,27003.600751,18793.304501,16867.693635,28729.487277,24038.201511,15483.377066,13587.594878,22279.847115,4337.884416,28185.632647,37026.813460,22646.366167,17928.280774,13788.646162,13726.741369


Transform dataframe to a numpy array

In [4]:
np_distance_matrix = df_distance_matrix.to_numpy()
#show the array
np_distance_matrix 

array([[18301.4739001 , 18368.666923  , 18581.507345  , ...,
        22232.2388169 , 13351.1654622 , 17360.9399864 ],
       [22264.1174263 ,  9331.90467368, 10136.5115516 , ...,
        25098.4981741 , 10922.4319139 ,  7431.12738605],
       [21041.7184025 ,  7198.05070753,  6578.35197004, ...,
        21153.0319565 , 12466.7158712 ,  9070.61835337],
       ...,
       [18633.3489679 ,  7680.33018417,  8182.91350331, ...,
        21428.8704091 ,  7293.40083847,  6330.32893737],
       [14838.7853061 , 11345.9591518 , 11426.6342395 , ...,
        18194.6201301 ,  6273.16345085, 10682.8955152 ],
       [22264.1174263 ,  9331.90467368, 10136.5115516 , ...,
        25098.4981741 , 10922.4319139 ,  7431.12738605]])

Take the first six rows only. We are testing if the optimization problem is formulated correctly

In [5]:
np_distance_matrix = np_distance_matrix[[0,1,2,3,4,5], :][:, [0,1,2]]
np_distance_matrix

array([[18301.4739001 , 18368.666923  , 18581.507345  ],
       [22264.1174263 ,  9331.90467368, 10136.5115516 ],
       [21041.7184025 ,  7198.05070753,  6578.35197004],
       [14402.7205813 , 17883.7461183 , 17864.5359133 ],
       [18863.277607  , 11661.0205502 , 10730.5450219 ],
       [22143.1657918 , 13981.6497957 , 14593.5129486 ]])

check dimensions

In [6]:
np.shape(np_distance_matrix)

(6, 3)

The depots are in alphabetical order. The csv with depot capacities needs to have the depots in alphabetical order as well!

IMPORTING CSV WITH GARAGE CAPACITIES

In [7]:
df_capacities = pd.read_csv("Garage Capacities.csv")
list(df_capacities.columns) 

['Name', 'Capacity']

In [8]:
df_capacities.drop(["Name"], axis=1, inplace= True)
df_capacities.head()

,Capacity
0,85
1,225
2,140
3,200
4,225


convert to a numpy array

In [9]:
np_depot_capacities = df_capacities.to_numpy()
# show the output
np_depot_capacities
# subset first three rows of capacities to test optimization problem
np_depot_capacities = np_depot_capacities[:3]      
np_depot_capacities

array([[ 85],
       [225],
       [140]])

In [10]:
np.shape(np_depot_capacities)

(3, 1)

We need to change this so that the capacities are all in one row. This way it will match the distance matrix array and
we can sum over the columns

In [11]:
# use flatten to collapse array into one dimension
np_depot_capacities = np_depot_capacities.flatten()
np_depot_capacities

array([ 85, 225, 140])

## Test Variable Used to check if Problem is set up Correctly
## If capacities < number of buses, optimization returns none

In [12]:
test_depot_capacities = np.array([52,47,68])

We need another array for the number of buses on each trip. This is necessary for the depot capacity constraint

In [13]:
# read in the csv
df_buses = pd.read_csv("bus-to-garage.csv")
# keep only the buses column
df_buses = df_buses[["standard_buses"]]
df_buses.head()

,standard_buses
0,4.0
1,4.0
2,26.0
3,26.0
4,11.0


In [14]:
np_buses = df_buses.to_numpy()
# subset first six rows of buses to test the optimization problem
np_buses = np_buses[:6]      
np_buses = np_buses.flatten()
np_buses

array([ 4.,  4., 26., 26., 11., 11.])

In [15]:
# we need to repeat each row 20 times, where 20 is the number of depots
# np.tile 'tiles' the array 20 times (20 rows of 340 values)
# .T transposes the array so that it is 340 rows with each row having the number of buses 
# of the respective trip repeated 20 times
buses = np.tile(np_buses,(3,1)).T
buses
#np.shape(buses)    # (340, 20)

array([[ 4.,  4.,  4.],
       [ 4.,  4.,  4.],
       [26., 26., 26.],
       [26., 26., 26.],
       [11., 11., 11.],
       [11., 11., 11.]])

# SETTING UP THE PROBLEM

### selection is a binary '0/1' matrix. 1 means bus is assigned to terminal

In [16]:
selection = cp.Variable(shape=buses.shape,boolean=True)

## Constraints

### trip can only be assigned to one depot (Row sums need to equal 1)

In [17]:
assignment_constraint = cp.sum(selection,axis=1) == 1

### depots cannot exceed capacities. Column sums <= depot capacity

In [18]:
capacity_constraint = cp.sum(cp.multiply(selection, buses), axis=0) <= test_depot_capacities

### Minimize dead mileage. Dead mileage is the product of the selection matrix and the (buses*distance)

In [19]:
cost = cp.sum(cp.multiply(cp.multiply(selection, buses), np_distance_matrix))

### Run the optimization problem

In [20]:
constraints = [assignment_constraint, capacity_constraint]

assign_problem = cp.Problem(cp.Minimize(cost),constraints)

assign_problem.solve(solver= cp.GUROBI, verbose = True)

print(selection.value)
print('Problem status: {}'.format(assign_problem.status))

Using license file /Library/gurobi900/gurobi.lic
Academic license - for non-commercial use only
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter QCPDual to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 9 rows, 18 columns and 36 nonzeros
Model fingerprint: 0x01db913b
Variable types: 0 continuous, 18 integer (18 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [4e+04, 5e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+01]
Found heuristic solution: objective 1070731.7270
Presolve removed 2 rows and 5 columns
Presolve time: 0.00s
Presolved: 7 rows, 13 columns, 25 nonzeros
Found heuristic solution: objective 927875.54362
Variable types: 0 continuous, 13 integer (13 binary)

Root relaxation: cutoff, 3 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj

test matrix multiplication!

In [21]:
array1 = np.array([[3,3,3],
          [1,2,1]])
array2 = np.array([[1,5,4],
          [1,2,1]])
test = cp.multiply(array1, array2)
print(test)
test2 = cp.sum(cp.multiply(array1, array2), axis=0)
print(test2)
test3 = array1*array2
test3

[[3. 3. 3.]
 [1. 2. 1.]] * [[1. 5. 4.]
 [1. 2. 1.]]
Sum([[3. 3. 3.]
 [1. 2. 1.]] * [[1. 5. 4.]
 [1. 2. 1.]], 0, False)


array([[ 3, 15, 12],
       [ 1,  4,  1]])